In [1]:
from pyspark.sql import * 
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName(".").getOrCreate()

In [3]:
loan_df = spark.read.format("csv").options(header="true",inferSchema="true").load("/user/test/loan.csv")

In [4]:
#loan_df.show(10,False)

In [5]:
spark

In [6]:
#Count of rows 

loan_df.count()

500

In [7]:
loan_df.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



#### 1. Demographic Insights

In [9]:
#Average Age: Calculate the average age of customers.

loan_df.select(avg("Age").alias("Average_Age")).show()

+-----------+
|Average_Age|
+-----------+
|     40.946|
+-----------+



In [10]:
#Gender Distribution: Count of male vs. female customers.
loan_df.groupBy("Gender").count().show()

+------+-----+
|Gender|count|
+------+-----+
|  MALE|  280|
|FEMALE|  220|
+------+-----+



In [11]:
#Marital Status Distribution: Percentage of married vs. single customers.
marital_per = loan_df.groupBy("Marital Status").count()
marital_per.select(sum("count").alias("Sum")).show()
marital_per.select(col("Marital Status"),col("count") / 500 * 100).show()

+---+
|Sum|
+---+
|500|
+---+

+--------------+---------------------+
|Marital Status|((count / 500) * 100)|
+--------------+---------------------+
|        SINGLE|                 29.2|
|       MARRIED|                 70.8|
+--------------+---------------------+



In [12]:
#Family Size Distribution: Average family size across different customer groups.

loan_df.groupBy("Occupation","Marital Status").agg(round(avg("Family Size"),2).alias("Average_Family_size")).orderBy("Occupation").show(truncate=False)

#Chat Gpt Answers
loan_df.groupBy("Occupation").agg(round(avg("Family Size"),2).alias("Average_Family_size")).orderBy("Occupation").show(truncate=False)
loan_df.groupBy("Marital Status").agg(round(avg("Family Size"),2).alias("Average_Family_size")).orderBy("Marital Status").show(truncate=False)



+---------------------+--------------+-------------------+
|Occupation           |Marital Status|Average_Family_size|
+---------------------+--------------+-------------------+
|ACCOUNT MANAGER      |MARRIED       |4.73               |
|ACCOUNTANT           |SINGLE        |5.5                |
|ACCOUNTANT           |MARRIED       |4.33               |
|AGRICULTURAL ENGINEER|SINGLE        |6.0                |
|AGRICULTURAL ENGINEER|MARRIED       |3.4                |
|AIRPORT OFFICER      |SINGLE        |4.0                |
|AIRPORT OFFICER      |MARRIED       |6.23               |
|ARMY                 |MARRIED       |5.0                |
|ARMY                 |SINGLE        |6.0                |
|ASSISTANT MANAGER    |MARRIED       |6.17               |
|ASSISTANT PROFESSOR  |MARRIED       |4.89               |
|BANK MANAGER         |SINGLE        |4.5                |
|BANK MANAGER         |MARRIED       |4.86               |
|BUSINESS             |SINGLE        |5.0               

#### 2. Financial Overview

In [14]:
#Average Monthly Income: Compute the average income of customers.
loan_df.select(avg("Income").alias("Average_Income")).show()

+-----------------+
|   Average_Income|
+-----------------+
|68339.49145299145|
+-----------------+



In [15]:
#Average Monthly Expenditure: Analyze spending patterns by calculating the average expenditure.

# Average Monthly Expenditure
loan_df.select(round(avg("Expenditure"),2).alias("Average_Monthly_Expenditure")).show()

# Average expenditure by Family size
loan_df.groupBy("Family Size").agg(round(avg("Expenditure"),2).alias("Average_Expenditure_By_Family_Size")).orderBy("Family Size").show()

#average expenditure by Loan Category
loan_df.groupBy("Loan Category").agg(round(avg("Expenditure"),2).alias("Average_Expenditure_By_Load_Cat")).orderBy("Loan Category").show()



# ChatGpt 
#average expenditure by gender 
loan_df.groupBy("Gender").agg(round(avg("Expenditure"),2).alias("Average_Expenditure_By_Gender")).show()

#average expenditure by Marital Status  
loan_df.groupBy("Marital Status").agg(round(avg("Expenditure"),2).alias("Average_Expenditure_By_MaritalStatues")).show()



+---------------------------+
|Average_Monthly_Expenditure|
+---------------------------+
|                   27533.18|
+---------------------------+

+-----------+----------------------------------+
|Family Size|Average_Expenditure_By_Family_Size|
+-----------+----------------------------------+
|          2|                          30383.33|
|          3|                          25790.01|
|          4|                          25633.58|
|          5|                          25842.11|
|          6|                           30232.2|
|          7|                           28854.2|
+-----------+----------------------------------+

+------------------+-------------------------------+
|     Loan Category|Average_Expenditure_By_Load_Cat|
+------------------+-------------------------------+
|       AGRICULTURE|                        30573.5|
|        AUTOMOBILE|                       26787.66|
|       BOOK STORES|                        21221.0|
|          BUILDING|                    

In [16]:
#Income-Expenditure Ratio: (Expenditure / Income) to identify over-leveraged customers.
Over_leveraged = loan_df.select("Customer_ID","Overdue"," Debt Record","Loan Amount","Loan Category","Expenditure","Income",
                                round(col("Expenditure")/col("Income"),2).alias("Ratio_E/I"))

#ChatGpt 
Over_leveraged.withColumn("over-leveraged_Status",when(col("Ratio_E/I")> 0.5,"Yes").otherwise("No")).show()


+-----------+-------+------------+-----------+------------------+-----------+------+---------+---------------------+
|Customer_ID|Overdue| Debt Record|Loan Amount|     Loan Category|Expenditure|Income|Ratio_E/I|over-leveraged_Status|
+-----------+-------+------------+-----------+------------------+-----------+------+---------+---------------------+
|    IB14001|      5|      42,898| 10,00,000 |           HOUSING|      22199| 50000|     0.44|                   No|
|    IB14008|      3|      33,999|     50,000|          SHOPPING|      19999| 51000|     0.39|                   No|
|    IB14012|      6|      20,876|     75,000|        TRAVELLING|      27675| 58450|     0.47|                   No|
|    IB14018|      7|      11,000|  6,00,000 |         GOLD LOAN|      12787| 45767|     0.28|                   No|
|    IB14022|      2|      43,898|  2,00,000 |        AUTOMOBILE|      11999| 43521|     0.28|                   No|
|    IB14024|      1|      50,000|     47,787|        AUTOMOBILE

In [17]:
#Top 5 Professions by Income: List the professions with the highest average income.

loan_df.groupBy("Occupation").agg(round(avg("Income")).alias("Average_Income")).orderBy(col("Average_Income").desc()).limit(5).show()


+--------------------+--------------+
|          Occupation|Average_Income|
+--------------------+--------------+
|     ACCOUNT MANAGER|      339097.0|
|        BANK MANAGER|       92191.0|
|AGRICULTURAL ENGI...|       82061.0|
|               PILOT|       81975.0|
|                ARMY|       78883.0|
+--------------------+--------------+



#### 3. Loan Analysis

In [19]:
#Most Popular Loan Categories: Count the number of customers per loan category.

loan_df.groupby("Loan Category").agg(count("Customer_ID").alias("Customer_Count")).orderBy(col("Customer_Count").desc()).show()

+------------------+--------------+
|     Loan Category|Customer_Count|
+------------------+--------------+
|         GOLD LOAN|            77|
|           HOUSING|            67|
|        AUTOMOBILE|            60|
|        TRAVELLING|            53|
|       RESTAURANTS|            41|
|COMPUTER SOFTWARES|            35|
|          SHOPPING|            35|
|          BUSINESS|            24|
|  EDUCATIONAL LOAN|            20|
|        RESTAURANT|            20|
|           DINNING|            14|
|       ELECTRONICS|            14|
|   HOME APPLIANCES|            14|
|       AGRICULTURE|            12|
|       BOOK STORES|             7|
|          BUILDING|             7|
+------------------+--------------+



In [20]:
#Average Loan Amount by Category: Compute the average loan amount for each loan type.
loan_df = loan_df.withColumn("Loan Amount",regexp_replace("Loan Amount","[,]",''))\
                    .withColumn("Loan Amount",col("Loan Amount").cast("integer"))

loan_df.groupBy("Loan Category").agg(round(avg("Loan Amount")).alias("Avg_Loan_Amount")).orderBy(col("Avg_Loan_Amount").desc()).show()

+------------------+---------------+
|     Loan Category|Avg_Loan_Amount|
+------------------+---------------+
|COMPUTER SOFTWARES|       994596.0|
|          BUSINESS|       973682.0|
|       AGRICULTURE|       965852.0|
|        AUTOMOBILE|       942383.0|
|         GOLD LOAN|       921967.0|
|  EDUCATIONAL LOAN|       919711.0|
|           HOUSING|       900688.0|
|        TRAVELLING|       690735.0|
|           DINNING|       654846.0|
|       ELECTRONICS|       640744.0|
|       RESTAURANTS|       609921.0|
|        RESTAURANT|       586462.0|
|   HOME APPLIANCES|       562852.0|
|          BUILDING|       541720.0|
|       BOOK STORES|       525950.0|
|          SHOPPING|       447012.0|
+------------------+---------------+



In [21]:
#Loan Utilization Rate: Ratio of Loan Amount to Income for all customers.
loan_df.select("Customer_ID","Loan Amount","Income",
                                round(col("Loan Amount")/col("Income"),2).alias("Ratio_LA/I_montly")).show()

+-----------+-----------+------+-----------------+
|Customer_ID|Loan Amount|Income|Ratio_LA/I_montly|
+-----------+-----------+------+-----------------+
|    IB14001|    1000000| 50000|             20.0|
|    IB14008|      50000| 51000|             0.98|
|    IB14012|      75000| 58450|             1.28|
|    IB14018|     600000| 45767|            13.11|
|    IB14022|     200000| 43521|              4.6|
|    IB14024|      47787| 34999|             1.37|
|    IB14025|    1209867| 46619|            25.95|
|    IB14027|      60676| 49999|             1.21|
|    IB14029|     399435| 45008|             8.87|
|    IB14031|      60999| 55999|             1.09|
|    IB14032|      35232| 60111|             0.59|
|    IB14034|      80660|  NULL|             NULL|
|    IB14037|      30999| 48099|             0.64|
|    IB14039|     987611| 45777|            21.57|
|    IB14041|     599934| 50999|            11.76|
|    IB14042|    1290929| 60111|            21.48|
|    IB14045|     167654| 40999

In [22]:
#High Loan Amount Customers: Count of customers with loans exceeding ₹10,00,000.
loan_df.filter(col("Loan Amount")>1000000).count()


130

#### 4. Risk Assessment

In [24]:
#Average Overdue Payments: Mean of the Overdue column.

loan_df.select(mean("Overdue")).show()


+------------+
|avg(Overdue)|
+------------+
|       4.974|
+------------+



In [25]:
#High-Risk Customers:Customers with Overdue > 5 or Debt Record > ₹50,000.
#loan_df.withColumn("Risk",when(((col("Overdue")>5)|(col(" Debt Record")>50000)),"High").otherwise("Low")).show()

loan_df.filter((col("Overdue")>5)|(col(" Debt Record")>50000)).show()

+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|       Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14012| 30|FEMALE|          DENTIST|        SINGLE|          3| 58450|      27675|            5|        TRAVELLING|      75000|      6|      20,876|               3|                 1|
|    IB14018| 29|  MALE|          TEACHER|       MARRIED|          5| 45767|      12787|            3|         GOLD LOAN|     600000|      7|      11,000|               0|                 4|
|    IB14025| 39|FEMALE|          TEACHER|   

In [26]:
#Customers with a high number of Returned Cheques or Dishonour of Bills.

loan_df.filter((col(" Dishonour of Bill")>5)|(col(" Returned Cheque")>5)).show()


+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|   Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|         HOUSING|    1000000|      5|      42,898|               6|                 9|
|    IB14025| 39|FEMALE|            TEACHER|       MARRIED|          6| 46619|      18675|            4|         HOUSING|    1209867|      8|      29,999|               6|                 8|
|    IB14029| 24|FEMALE|            TEACHER| 

In [27]:
#Debt Burden Ratio: (Debt Record / Loan Amount).

loan_df = loan_df.withColumn(" Debt Record",regexp_replace(" Debt Record","[,]",''))\
                    .withColumn(" Debt Record",col(" Debt Record").cast("integer"))

loan_df.select(round(col("Loan Amount")/col(" Debt Record")).alias("Debt_Burden_Ratio")).show()


+-----------------+
|Debt_Burden_Ratio|
+-----------------+
|             23.0|
|              1.0|
|              4.0|
|             55.0|
|              5.0|
|              1.0|
|             40.0|
|              5.0|
|              8.0|
|             NULL|
|              1.0|
|             18.0|
|              3.0|
|             25.0|
|             67.0|
|             72.0|
|             37.0|
|             12.0|
|             53.0|
|             43.0|
+-----------------+
only showing top 20 rows



#### 5. Customer Behavior

In [29]:
#Use Frequency Analysis: Average usage frequency across all customers.
loan_df.select(avg("Use Frequency").alias("Average_usage_frequency")).show()


+-----------------------+
|Average_usage_frequency|
+-----------------------+
|                   5.33|
+-----------------------+



In [30]:
#Top 5 Customer Segments by Use Frequency: Group by Loan Category or Occupation and compute average usage frequency.

loan_df.groupBy("Loan Category").agg(round(avg("Use Frequency")).alias("Average_usage_frequency")).orderBy(col("Average_usage_frequency").desc()).show()

loan_df.groupBy("Occupation").agg(round(avg("Use Frequency")).alias("Average_usage_frequency")).orderBy(col("Average_usage_frequency").desc()).show()

+------------------+-----------------------+
|     Loan Category|Average_usage_frequency|
+------------------+-----------------------+
|          BUILDING|                    7.0|
|        TRAVELLING|                    6.0|
|       AGRICULTURE|                    6.0|
|COMPUTER SOFTWARES|                    6.0|
|       ELECTRONICS|                    6.0|
|        RESTAURANT|                    6.0|
|           HOUSING|                    5.0|
|         GOLD LOAN|                    5.0|
|  EDUCATIONAL LOAN|                    5.0|
|        AUTOMOBILE|                    5.0|
|          BUSINESS|                    5.0|
|           DINNING|                    5.0|
|          SHOPPING|                    5.0|
|       RESTAURANTS|                    5.0|
|   HOME APPLIANCES|                    5.0|
|       BOOK STORES|                    4.0|
+------------------+-----------------------+

+-----------------+-----------------------+
|       Occupation|Average_usage_frequency|
+----------

In [31]:
#Correlations:
#Correlate Income and Expenditure.
loan_df.stat.corr("Income","Expenditure")

0.022779656082440435

In [32]:

#Correlate Overdue with Debt Record.
loan_df.stat.corr("Overdue"," Debt Record")

0.050553055880314626

#### 6. Outlier Analysis

In [34]:
#Identify outliers in:
#Income: Customers earning exceptionally high or low.
loan_df.select(max("Income")).show()
loan_df.select(min("Income")).show()


+-----------+
|max(Income)|
+-----------+
|     930000|
+-----------+

+-----------+
|min(Income)|
+-----------+
|      28366|
+-----------+



In [35]:
#Expenditure: Unusually high spenders relative to income.

#Chat GPT 
df = loan_df.withColumn("E_to_I",col("Expenditure")/col("Income"))

stats = df.select(round(avg("E_to_I"),2).alias("Mean_ratio"),round(stddev("E_to_I"),2).alias("stddev_ratio")).collect()

mean_ratio = stats[0]["Mean_ratio"]
stddev_ratio = stats[0]["stddev_ratio"]

threshold = mean_ratio +2 * stddev_ratio

df_flagged = df.withColumn(
    "Unusually_High_Spender",
    when(col("E_to_I") > threshold, "Yes").otherwise("No")
)

df_flagged.show()



+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+-------------------+----------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|             E_to_I|Unusually_High_Spender|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+-------------------+----------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING|    1000000|      5|       42898|               6|                 9|            0.44398|                    No|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRI

In [36]:
#Loan Amounts: Loans significantly higher than the average.

loan_df.select(round(avg("Loan Amount"))).show()

loan_df.filter(col("Loan Amount")>797053.0).count()

+--------------------------+
|round(avg(Loan Amount), 0)|
+--------------------------+
|                  797053.0|
+--------------------------+



201

#### 7. General Analysis Questions


In [38]:
#What is the average age of customers?
loan_df.select(avg("Age")).show()

+--------+
|avg(Age)|
+--------+
|  40.946|
+--------+



In [39]:
#How many customers are there in total?
loan_df.select("Customer_ID").distinct().count()

499

In [40]:
#How many unique occupations are there in the dataset?
loan_df.select("Occupation").distinct().count()

39

In [41]:
#What is the total loan amount across all customers?
loan_df.select(sum("Loan amount").alias("Total_Loan_Amount")).show()

+-----------------+
|Total_Loan_Amount|
+-----------------+
|        398526449|
+-----------------+



In [42]:
#What is the average expenditure of customers?
loan_df.select(round(avg("Expenditure")).alias("Average_Expenditure")).show()

+-------------------+
|Average_Expenditure|
+-------------------+
|            27533.0|
+-------------------+



In [43]:
#Which marital status group has the highest total loan amount?
loan_df.groupBy("Marital Status").agg(sum("Loan Amount").alias("Total_Loan_Amount")).orderBy(col("Total_Loan_Amount").desc()).limit(1).show()


+--------------+-----------------+
|Marital Status|Total_Loan_Amount|
+--------------+-----------------+
|       MARRIED|        286407764|
+--------------+-----------------+



In [44]:
#How many customers have a loan amount greater than 1,000,000?
loan_df.filter(col("Loan Amount")>1000000).count()


130

In [45]:
#Which loan category has the most customers?
loan_df.groupBy("Occupation").count().orderBy(col("count").desc()).limit(1).show()

+----------+-----+
|Occupation|count|
+----------+-----+
|   TEACHER|   63|
+----------+-----+



In [46]:
#What is the total debt record count for each marital status (Single vs. Married)?
#Chat gpt
loan_df.groupBy("Marital Status").agg(sum(" Debt Record").alias("total debt record")).show()

+--------------+-----------------+
|Marital Status|total debt record|
+--------------+-----------------+
|        SINGLE|          6730508|
|       MARRIED|         13149192|
+--------------+-----------------+



In [47]:
#What is the average family size for male and female customers?
loan_df.groupBy("Gender").agg(round(avg("Family Size")).alias("Average_Family_Size")).show()

+------+-------------------+
|Gender|Average_Family_Size|
+------+-------------------+
|  MALE|                5.0|
|FEMALE|                4.0|
+------+-------------------+



In [48]:
#Which occupation has the highest average loan amount?

loan_df.groupBy("Occupation").agg(round(avg("Loan Amount")).alias("Average_Loan_Amount")).orderBy(col("Average_Loan_Amount").desc()).limit(1).show()


+-----------------+-------------------+
|       Occupation|Average_Loan_Amount|
+-----------------+-------------------+
|CORPORATE OFFICER|          5230555.0|
+-----------------+-------------------+



In [49]:
#How many customers have a 'Returned Cheque' value greater than 5?

loan_df.filter(col(" Returned Cheque") > 5).count()

156

#### 8. Gender-Based Analysis

In [51]:
#How many male customers have a loan amount greater than 500,000?
df = loan_df.filter(col("Gender") == "MALE")
df.filter(col("Loan Amount")> 500000).count()


193

In [52]:
#What is the average loan amount for male customers compared to female customers?
df = loan_df.groupBy("Gender").agg(round(avg("Loan Amount"),2).alias("Average_Loan_Amount"))
df.select(sum("Average_Loan_Amount")).show()
df.select(col("Average_Loan_Amount")/1607180.83).show()



+------------------------+
|sum(Average_Loan_Amount)|
+------------------------+
|              1607180.83|
+------------------------+

+----------------------------------+
|(Average_Loan_Amount / 1607180.83)|
+----------------------------------+
|                0.4661025729133417|
|                0.5338974270866583|
+----------------------------------+



In [53]:
#How many female customers have overdue counts greater than 5?

loan_df.filter((col("Gender") == "FEMALE") & (col("Overdue")>5)).count()

91

In [54]:
#What is the total loan amount for male customers in the 'TRAVELLING' loan category?
df = loan_df.groupBy("Loan Category","Gender").agg(sum("Loan Amount"))
df.filter((col("Loan Category") == "TRAVELLING")& (col("Gender")=="MALE")).show()

+-------------+------+----------------+
|Loan Category|Gender|sum(Loan Amount)|
+-------------+------+----------------+
|   TRAVELLING|  MALE|        22323627|
+-------------+------+----------------+



#### 9. Loan Category Analysis:

In [56]:
#What is the total loan amount for each loan category (e.g., HOUSING, SHOPPING, AUTOMOBILE)?

df = loan_df.groupby("Loan Category").agg(sum("Loan Amount")).orderBy(col("sum(Loan Amount)").desc())
df.show()

+------------------+----------------+
|     Loan Category|sum(Loan Amount)|
+------------------+----------------+
|         GOLD LOAN|        70991425|
|           HOUSING|        60346129|
|        AUTOMOBILE|        56542964|
|        TRAVELLING|        36608973|
|COMPUTER SOFTWARES|        34810861|
|       RESTAURANTS|        25006754|
|          BUSINESS|        23368358|
|  EDUCATIONAL LOAN|        18394223|
|          SHOPPING|        15645414|
|        RESTAURANT|        11729243|
|       AGRICULTURE|        11590221|
|           DINNING|         9167850|
|       ELECTRONICS|         8970419|
|   HOME APPLIANCES|         7879927|
|          BUILDING|         3792037|
|       BOOK STORES|         3681651|
+------------------+----------------+



In [57]:
#What is the average loan amount for customers in the 'GOLD LOAN' category?

df = loan_df.groupBy("Loan Category").agg(round(avg("Loan Amount"),1))

df.filter(col("Loan Category") == "GOLD LOAN").show()


+-------------+--------------------------+
|Loan Category|round(avg(Loan Amount), 1)|
+-------------+--------------------------+
|    GOLD LOAN|                  921966.6|
+-------------+--------------------------+



In [58]:
#Which loan category has the most customers with overdue values greater than 5?

loan_df.filter(col("Overdue")>5)\
                    .groupBy("Loan Category")\
                    .agg(count("Customer_ID").alias("Customer_Count"))\
                    .orderBy(col("Customer_Count").desc()).show()


+------------------+--------------+
|     Loan Category|Customer_Count|
+------------------+--------------+
|         GOLD LOAN|            38|
|           HOUSING|            31|
|        TRAVELLING|            25|
|        AUTOMOBILE|            20|
|COMPUTER SOFTWARES|            19|
|          SHOPPING|            18|
|       RESTAURANTS|            15|
|          BUSINESS|            13|
|        RESTAURANT|            13|
|  EDUCATIONAL LOAN|             8|
|       ELECTRONICS|             8|
|       AGRICULTURE|             7|
|           DINNING|             5|
|   HOME APPLIANCES|             5|
|       BOOK STORES|             3|
|          BUILDING|             3|
+------------------+--------------+



#### 10. Family Size and Marital Status Analysis


In [60]:
#How does the family size of single customers compare to married customers?

loan_df.groupBy("Marital Status").agg(round(avg("Family Size"),2)).show()


+--------------+--------------------------+
|Marital Status|round(avg(Family Size), 2)|
+--------------+--------------------------+
|        SINGLE|                      4.21|
|       MARRIED|                      4.69|
+--------------+--------------------------+



In [61]:
#What is the total loan amount for customers with a family size greater than 5?

loan_df.groupBy("Family Size").agg(sum("Loan Amount")).filter(col("Family Size")>5).show()

+-----------+----------------+
|Family Size|sum(Loan Amount)|
+-----------+----------------+
|          6|        65084466|
|          7|        55001865|
+-----------+----------------+



In [62]:
#How many married customers have a family size of 4 or more?

df = loan_df.groupBy("Marital Status","Family Size").count()
df.filter(col("Marital Status") == "MARRIED").filter(col("Family Size")>4).count()

3

#### 11. Income and Expenditure Analysis


In [64]:
#What is the average income for customers who are married?

loan_df.groupBy("Marital Status").agg(round(avg("Income"),2).alias("Average_Income"))\
        .filter(col("Marital Status")== "MARRIED").show()

+--------------+--------------+
|Marital Status|Average_Income|
+--------------+--------------+
|       MARRIED|      71465.58|
+--------------+--------------+



In [65]:
#Which customer has the highest income and what is their loan amount?

loan_df.select("Customer_ID","Loan Amount","Income").orderBy(col("Income").desc()).limit(1).show()

+-----------+-----------+------+
|Customer_ID|Loan Amount|Income|
+-----------+-----------+------+
|    IBI4157|     679040|930000|
+-----------+-----------+------+



In [66]:
#How many customers have an income greater than 50,000 and an expenditure greater than 20,000?
loan_df.filter( (col("Income") > 50000) & (col("Expenditure")>20000) ).count()


225

In [67]:
#What is the total income and expenditure for all customers combined?
loan_df.select(sum("Income"),sum("Expenditure")).show()


+-----------+----------------+
|sum(Income)|sum(Expenditure)|
+-----------+----------------+
|   31982882|        13243460|
+-----------+----------------+



#### 12. Specific Customer Query

In [69]:
#What are the details of the customer with the highest overdue count?

loan_df.orderBy(col("Overdue").desc()).limit(1).show()


+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14029| 24|FEMALE|   TEACHER|        SINGLE|          3| 45008|      17454|            4|   AUTOMOBILE|     399435|      9|       51987|               4|                 7|
+-----------+---+------+----------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+



In [70]:
#Which customer has the lowest family size, and what is their loan amount?

df = loan_df.orderBy(col("Family Size").desc()).limit(1)
df.select("Loan Amount").show()


+-----------+
|Loan Amount|
+-----------+
|     987611|
+-----------+



In [71]:
#How many customers have a debt record greater than 5?

loan_df.filter(col(" Debt Record")>5).count()

496

In [72]:
#Which occupation group has the highest average overdue count?

loan_df.groupBy("Occupation").agg(round(avg("Overdue"),1)).orderBy(col("round(avg(Overdue), 1)").desc()).limit(1).show()

+------------+----------------------+
|  Occupation|round(avg(Overdue), 1)|
+------------+----------------------+
|PUBLIC WORKS|                   8.7|
+------------+----------------------+



#### 13. Aggregation and Grouping Questions

In [74]:
#What is the total loan amount per marital status?

loan_df.groupBy("Marital Status").agg(sum("Loan Amount")).show()


+--------------+----------------+
|Marital Status|sum(Loan Amount)|
+--------------+----------------+
|        SINGLE|       112118685|
|       MARRIED|       286407764|
+--------------+----------------+



In [75]:
#What is the average family size per occupation?

loan_df.groupBy("Occupation").agg(round(avg("Family Size"),1)).show()



+--------------------+--------------------------+
|          Occupation|round(avg(Family Size), 1)|
+--------------------+--------------------------+
|      CIVIL ENGINEER|                       4.8|
|     FIRE DEPARTMENT|                       4.5|
|          ACCOUNTANT|                       5.0|
|        BANK MANAGER|                       4.8|
|      SYSTEM OFFICER|                       5.8|
|           NUTRITION|                       6.0|
|           DIETICIAN|                       3.8|
|               CLERK|                       4.3|
|   SOFTWARE ENGINEER|                       4.1|
|AGRICULTURAL ENGI...|                       4.4|
|   ASSISTANT MANAGER|                       6.2|
|             TEACHER|                       4.6|
| ASSISTANT PROFESSOR|                       4.9|
|     SYSTEM ENGINEER|                       4.7|
| CHARTERED APPRAISER|                       4.0|
|                NAVY|                       4.3|
|              POLICE|                       4.2|


In [76]:
#What is the total debt record count by loan category?
loan_df.groupBy("Loan Category").agg(sum(" Debt Record")).show()

+------------------+-----------------+
|     Loan Category|sum( Debt Record)|
+------------------+-----------------+
|           HOUSING|          2685248|
|        TRAVELLING|          2105826|
|       BOOK STORES|           259858|
|       AGRICULTURE|           529396|
|         GOLD LOAN|          3360447|
|  EDUCATIONAL LOAN|           714777|
|        AUTOMOBILE|          2289605|
|          BUSINESS|          1022936|
|COMPUTER SOFTWARES|          1106062|
|           DINNING|           458179|
|          SHOPPING|          1598401|
|       RESTAURANTS|          1628403|
|       ELECTRONICS|           473985|
|          BUILDING|           146894|
|        RESTAURANT|           943240|
|   HOME APPLIANCES|           556443|
+------------------+-----------------+



In [163]:
#What is the sum of loan amounts for customers who have an expenditure greater than 20,000?

df = loan_df.filter(col("Expenditure")>20000)
df.select(sum("Loan Amount").alias("Total_Laon_Amount")).show()


+-----------------+
|Total_Laon_Amount|
+-----------------+
|        284854416|
+-----------------+



In [165]:
#How many customers have a loan amount greater than 100,000 and are in the 'SINGLE' marital status group?
loan_df.filter((col("Loan Amount")>100000) & (col("Marital Status") == "SINGLE")).count()

139